In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
df = pd.read_csv('Stock Prices Data Set.csv')
df

,symbol,date,open,high,low,close,volume
0,AAL,2014-01-02,25.0700,25.8200,25.0600,25.3600,8998943
1,AAPL,2014-01-02,79.3828,79.5756,78.8601,79.0185,58791957
2,AAP,2014-01-02,110.3600,111.8800,109.2900,109.7400,542711
3,ABBV,2014-01-02,52.1200,52.3300,51.5200,51.9800,4569061
4,ABC,2014-01-02,70.1100,70.2300,69.4800,69.8900,1148391
...,...,...,...,...,...,...,...
497467,XYL,2017-12-29,68.5300,68.8000,67.9200,68.2000,1046677
497468,YUM,2017-12-29,82.6400,82.7100,81.5900,81.6100,1347613
497469,ZBH,2017-12-29,121.7500,121.9500,120.6200,120.6700,1023624
497470,ZION,2017-12-29,51.2800,51.5500,50.8100,50.8300,1261916


### Handle missing data

In [8]:
df.shape

(497472, 7)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497472 entries, 0 to 497471
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   symbol  497472 non-null  object 
 1   date    497472 non-null  object 
 2   open    497461 non-null  float64
 3   high    497464 non-null  float64
 4   low     497464 non-null  float64
 5   close   497472 non-null  float64
 6   volume  497472 non-null  int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 26.6+ MB


In [14]:
# finding missing value
df.isnull().sum()

symbol     0
date       0
open      11
high       8
low        8
close      0
volume     0
dtype: int64

In [15]:
# percentage of missing value
df.isnull().sum()/df.shape[0] * 100

symbol    0.000000
date      0.000000
open      0.002211
high      0.001608
low       0.001608
close     0.000000
volume    0.000000
dtype: float64

The percentage of missing data is very low, we can just drop them

In [19]:
# Drop rows with any missing values
df = df.dropna()
df.isnull().sum()

symbol    0
date      0
open      0
high      0
low       0
close     0
volume    0
dtype: int64

### Detect and remove outliers.

In [35]:
# Check which column has the outliers
columns_to_check = ['open', 'high', 'low', 'close', 'volume']

for col in columns_to_check:
    if col in df.columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        
        print(f"{col}:")
        print(f"  Outliers: {len(outliers)} ({len(outliers)/len(df)*100:.1f}%)")
        print(f"  Range: {df[col].min():.2f} to {df[col].max():.2f}")
        print()

open:
  Outliers: 30313 (6.1%)
  Range: 1.62 to 2044.00

high:
  Outliers: 30393 (6.1%)
  Range: 1.69 to 2067.99

low:
  Outliers: 30239 (6.1%)
  Range: 1.50 to 2035.11

close:
  Outliers: 30337 (6.1%)
  Range: 1.59 to 2049.00

volume:
  Outliers: 47680 (9.6%)
  Range: 101.00 to 618237630.00



In [37]:
# Removing outliers
def remove_outliers_iqr(df, columns):
    """Remove outliers using IQR method"""
    df_clean = df.copy()
    
    for col in columns:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Remove outliers
        df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]
        
    return df_clean

# Remove outliers from all numeric columns
numeric_columns = ['open', 'high', 'low', 'close', 'volume']
df = remove_outliers_iqr(df, numeric_columns)

print(f"Original dataset: {len(df):,} rows")
print(f"After removing outliers: {len(df_no_outliers):,} rows")
print(f"Removed: {len(df) - len(df_no_outliers):,} rows ({((len(df) - len(df_no_outliers))/len(df)*100):.1f}%)")

Original dataset: 408,752 rows
After removing outliers: 408,752 rows
Removed: 0 rows (0.0%)


In [38]:
df

,symbol,date,open,high,low,close,volume
0,AAL,2014-01-02,25.07,25.82,25.06,25.36,8998943
2,AAP,2014-01-02,110.36,111.88,109.29,109.74,542711
3,ABBV,2014-01-02,52.12,52.33,51.52,51.98,4569061
4,ABC,2014-01-02,70.11,70.23,69.48,69.89,1148391
5,ABT,2014-01-02,38.09,38.40,38.00,38.23,4967472
...,...,...,...,...,...,...,...
497467,XYL,2017-12-29,68.53,68.80,67.92,68.20,1046677
497468,YUM,2017-12-29,82.64,82.71,81.59,81.61,1347613
497469,ZBH,2017-12-29,121.75,121.95,120.62,120.67,1023624
497470,ZION,2017-12-29,51.28,51.55,50.81,50.83,1261916


### Convert categorical variables into numerical format using one-hot encoding or label encoding.

In [41]:
# One-hot encoding using pandas
def apply_one_hot_encoding(df, columns_to_encode):
    """Apply one-hot encoding to specified columns"""
    df_encoded = df.copy()
    
    for col in columns_to_encode:
        # Get dummies (one-hot encoding)
        dummies = pd.get_dummies(df_encoded[col], prefix=col, prefix_sep='_')
        
        # Concatenate with original dataframe
        df_encoded = pd.concat([df_encoded, dummies], axis=1)
        
        # Drop original column
        df_encoded = df_encoded.drop(col, axis=1)
    
    return df_encoded

# Apply one-hot encoding to 'symbol' column
# Note: Be careful with 'symbol' if you have many unique stocks (creates many columns)
df_onehot = apply_one_hot_encoding(df, ['symbol'])

print(f"Original shape: {df.shape}")
print(f"After one-hot encoding: {df_onehot.shape}")
print(f"New columns created: {df_onehot.shape[1] - df.shape[1]}")

Original shape: (408752, 7)
After one-hot encoding: (408752, 492)
New columns created: 485


In [44]:
df_onehot

,date,open,high,low,close,volume,symbol_A,symbol_AAL,symbol_AAP,symbol_ABBV,...,symbol_XL,symbol_XLNX,symbol_XOM,symbol_XRAY,symbol_XRX,symbol_XYL,symbol_YUM,symbol_ZBH,symbol_ZION,symbol_ZTS
0,2014-01-02,25.07,25.82,25.06,25.36,8998943,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2014-01-02,110.36,111.88,109.29,109.74,542711,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3,2014-01-02,52.12,52.33,51.52,51.98,4569061,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
4,2014-01-02,70.11,70.23,69.48,69.89,1148391,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,2014-01-02,38.09,38.40,38.00,38.23,4967472,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497467,2017-12-29,68.53,68.80,67.92,68.20,1046677,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
497468,2017-12-29,82.64,82.71,81.59,81.61,1347613,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
497469,2017-12-29,121.75,121.95,120.62,120.67,1023624,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
497470,2017-12-29,51.28,51.55,50.81,50.83,1261916,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


### Normalize or standardize numerical data.

In [49]:
from sklearn.preprocessing import StandardScaler

# Select numerical columns
numerical_columns = ['open', 'high', 'low', 'close', 'volume']

# Create and fit the scaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df[numerical_columns])

# Convert back to DataFrame
df_standardized = df.copy()
df_standardized[numerical_columns] = scaled_data


print(df_standardized[numerical_columns].describe())

               open          high           low         close        volume
count  4.087520e+05  4.087520e+05  4.087520e+05  4.087520e+05  4.087520e+05
mean   4.728237e-17  5.501443e-16  1.107520e-15  1.707728e-16  1.042298e-16
std    1.000001e+00  1.000001e+00  1.000001e+00  1.000001e+00  1.000001e+00
min   -2.021297e+00 -2.023280e+00 -2.019795e+00 -2.021948e+00 -1.283569e+00
25%   -7.767477e-01 -7.768264e-01 -7.775250e-01 -7.771261e-01 -7.498907e-01
50%   -1.438663e-01 -1.440718e-01 -1.437892e-01 -1.440471e-01 -3.226070e-01
75%    6.178968e-01  6.171166e-01  6.181076e-01  6.179642e-01  4.597443e-01
max    2.930316e+00  2.914746e+00  2.722467e+00  2.682710e+00  3.346585e+00
